In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import requests
import folium
import time
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import plotly.io as pio
import json


In [2]:
df = pd.read_csv('fulton_home_value.csv')
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,3/31/2013,...,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020
0,71038,162,30318,Zip,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,118936,...,255294,256222,257309,258140,258806,259562,261119,262960,265523,267538
1,71050,262,30331,Zip,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,91606,...,174756,175331,176303,177152,177879,178782,180166,182066,184377,186278
2,70808,301,30022,Zip,GA,GA,Johns Creek,Atlanta-Sandy Springs-Roswell,Fulton County,314143,...,432271,432146,431872,431638,431648,432207,433749,434649,436290,437304
3,70791,302,30004,Zip,GA,GA,Milton,Atlanta-Sandy Springs-Roswell,Fulton County,321086,...,454907,455490,456018,456317,456493,457159,458781,460420,462150,463781
4,70854,602,30075,Zip,GA,GA,Roswell,Atlanta-Sandy Springs-Roswell,Fulton County,315211,...,446155,446312,446489,446915,446843,446616,447157,447829,449612,450374


In [3]:
df.columns

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName', '3/31/2013', '4/30/2013',
       '5/31/2013', '6/30/2013', '7/31/2013', '8/31/2013', '9/30/2013',
       '10/31/2013', '11/30/2013', '12/31/2013', '1/31/2014', '2/28/2014',
       '3/31/2014', '4/30/2014', '5/31/2014', '6/30/2014', '7/31/2014',
       '8/31/2014', '9/30/2014', '10/31/2014', '11/30/2014', '12/31/2014',
       '1/31/2015', '2/28/2015', '3/31/2015', '4/30/2015', '5/31/2015',
       '6/30/2015', '7/31/2015', '8/31/2015', '9/30/2015', '10/31/2015',
       '11/30/2015', '12/31/2015', '1/31/2016', '2/29/2016', '3/31/2016',
       '4/30/2016', '5/31/2016', '6/30/2016', '7/31/2016', '8/31/2016',
       '9/30/2016', '10/31/2016', '11/30/2016', '12/31/2016', '1/31/2017',
       '2/28/2017', '3/31/2017', '4/30/2017', '5/31/2017', '6/30/2017',
       '7/31/2017', '8/31/2017', '9/30/2017', '10/31/2017', '11/30/2017',
       '12/31/2017', '1/31/2018', '2/28/2018', '3

In [4]:
df.drop(['RegionID', 'SizeRank', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName'], axis =1, inplace = True)

In [5]:
df.set_index('RegionName', inplace = True)

In [6]:
df_trans = df.T.reset_index()
df_trans = df_trans.rename(columns={'index': 'Date'})

In [7]:
df_trans.Date = pd.to_datetime(df_trans.Date)

In [8]:
new_col = {}
for i in df_trans.columns[1:37]:
    new_col[i] = '{}'.format(i)

df_trans = df_trans.rename(columns = new_col)

In [173]:
data = []
tracex = []
zipcodes = list(df_trans.iloc[:,1:37])

for i in range(0, len(zipcodes)):
    tracex = go.Scatter(
        x = df_trans['Date'],
        y = df_trans[zipcodes[i]],
        mode = 'lines',
        name = zipcodes[i] )
    data.append(tracex)
    
fig = go.Figure(data = data)
fig.update_xaxes(title = '<b>Year</b>')
fig.update_yaxes(title = '<b>Average House Price in $</b>')
fig.update_layout(height = 600, width = 1200)

fig.show()
    

In [10]:
df_pct = df_trans.iloc[:,1:37].pct_change()
df_pct = df_pct.applymap(lambda x: x * 100)


In [11]:
df_pct['Date'] = df_trans.Date
df_pct

RegionName,30318,30331,30022,30004,30075,30309,30328,30324,30076,30350,...,30354,30317,30326,30337,30268,30313,30363,30303,30336,Date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-03-31
1,0.997175,0.330764,0.780536,0.814424,0.907329,1.120733,0.833885,0.696903,1.009980,0.893980,...,0.640594,1.824854,0.850237,0.688185,0.689855,1.622379,1.895905,0.424715,-0.065020,2013-04-30
2,1.595045,0.592978,0.859142,0.842444,1.027129,1.382798,0.879247,0.886896,1.217901,0.933671,...,0.481468,2.319987,0.984811,-1.216653,0.918608,2.133371,1.904383,0.981949,-0.458606,2013-05-31
3,1.450368,1.026456,0.683338,0.854093,0.657874,1.281692,0.710347,0.769567,0.967523,0.752740,...,0.644836,1.643458,1.142980,-0.103354,1.827129,1.785584,2.282736,0.625000,0.634485,2013-06-30
4,1.647713,1.899297,0.924736,1.023644,0.813104,1.535862,0.949665,1.025194,1.022135,0.927420,...,0.952181,1.527923,1.642514,-0.592986,2.591417,1.255366,2.268923,0.754272,1.257010,2013-07-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,0.292111,0.507648,0.129504,0.145895,-0.050801,0.024483,0.158936,0.073613,0.152349,0.076093,...,0.755570,-0.287679,-0.171798,0.218358,0.282845,0.132437,-0.207276,0.347463,0.517231,2019-11-30
81,0.599857,0.774127,0.356773,0.354800,0.121133,0.406887,0.438960,0.240035,0.349316,0.308168,...,0.793480,-0.089205,0.384797,1.084097,0.567906,0.378653,0.264418,0.732341,1.017615,2019-12-31
82,0.705043,1.054583,0.207493,0.357251,0.150283,0.476936,0.345919,0.234689,0.175495,0.508414,...,1.157621,-0.020677,0.276388,1.797965,0.859773,0.296467,0.545117,1.346042,1.060413,2020-01-31
83,0.974673,1.269320,0.377546,0.375744,0.398143,0.670193,0.412234,0.314604,0.343818,0.671950,...,1.479265,0.039717,0.588491,2.226354,1.088100,0.303360,0.732375,1.997003,1.429152,2020-02-29


In [12]:
df_pct.columns.shape

(37,)

In [174]:
data = []
tracex = []
zipcodes = list(df_pct.iloc[:,0:36])

for i in range(0, len(zipcodes)):
    tracex = go.Scatter(
        x = df_pct['Date'],
        y = df_pct[zipcodes[i]],
        mode = 'lines',
        name = '{}'.format(zipcodes[i]))
    data.append(tracex)
    
fig = go.Figure(data = data)
fig.update_xaxes(title = '<b>Year</b>')
fig.update_yaxes(title = '<b>Percent Change</b>')
fig.update_layout(height = 600, width = 1200)

fig.show()

In [14]:
top_ten = [i for i in df_pct.sum().sort_values(ascending = False)[0:10].keys()]

In [15]:
top_ten

['30310',
 '30314',
 '30311',
 '30315',
 '30344',
 '30354',
 '30318',
 '30291',
 '30316',
 '30331']

In [16]:
top_ten.insert(0,'Date')

top_ten

['Date',
 '30310',
 '30314',
 '30311',
 '30315',
 '30344',
 '30354',
 '30318',
 '30291',
 '30316',
 '30331']

In [17]:
df_pct_top = df_pct.loc[:,top_ten]
df_pct_top

RegionName,Date,30310,30314,30311,30315,30344,30354,30318,30291,30316,30331
0,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-04-30,2.843794,1.135749,0.522308,1.188673,1.920779,0.640594,0.997175,2.478154,0.966876,0.330764
2,2013-05-31,3.025930,0.453555,0.900070,1.218639,0.785486,0.481468,1.595045,1.577041,1.253477,0.592978
3,2013-06-30,1.533000,0.609239,0.833901,0.305412,0.300986,0.644836,1.450368,1.610795,1.387291,1.026456
4,2013-07-31,1.818723,-0.133260,0.881369,0.357366,0.537829,0.952181,1.647713,1.595046,1.495192,1.899297
...,...,...,...,...,...,...,...,...,...,...,...
80,2019-11-30,0.531485,1.721968,0.877729,0.644817,0.476999,0.755570,0.292111,0.637275,0.028454,0.507648
81,2019-12-31,0.656496,2.231141,1.392450,0.837623,0.807399,0.793480,0.599857,0.892481,0.290023,0.774127
82,2020-01-31,0.881137,2.632278,0.987561,1.308139,0.877488,1.157621,0.705043,1.180590,0.294425,1.054583
83,2020-02-29,0.975771,3.126150,0.968539,1.483401,1.092924,1.479265,0.974673,1.313090,0.358421,1.269320


In [175]:
data = []
tracex = []
zipcodes = list(df_pct_top.iloc[:,1:len(top_ten)])

for i in range(0, len(zipcodes)):
    tracex = go.Scatter(
        x = df_pct_top['Date'],
        y = df_pct_top[zipcodes[i]],
        mode = 'lines',
        name = '{}'.format(zipcodes[i]))
    data.append(tracex)
    
fig = go.Figure(data = data)
fig.update_xaxes(title = '<b>Year</b>')
fig.update_yaxes(title = '<b>Percent Change</b>')
fig.update_layout(height = 600, width = 1200)

fig.show()

In [19]:
df_trans_top = df_trans.loc[:,top_ten]

In [185]:
df_trans_top.Date

0    2013-03-31
1    2013-04-30
2    2013-05-31
3    2013-06-30
4    2013-07-31
        ...    
80   2019-11-30
81   2019-12-31
82   2020-01-31
83   2020-02-29
84   2020-03-31
Name: Date, Length: 85, dtype: datetime64[ns]

In [205]:
data = []
tracex = []
zipcodes = list(df_trans_top.iloc[:,1:len(top_ten)])

for i in range(0, len(zipcodes)):
    tracex = go.Scatter(
        x = df_trans_top['Date'],
        y = df_trans_top[zipcodes[i]],
        mode = 'lines',
        name = '{}'.format(zipcodes[i]))
    data.append(tracex)
    
fig = go.Figure(data = data)
fig.update_xaxes(title = '<b>Year</b>')
fig.update_yaxes(title = '<b>Average House Price in $</b>')
fig.update_layout(height = 600, width = 1200,
                 shapes=
        # 1st highlight during Feb 4 - Feb 6
        [dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="paper",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=.65,
            y0=0,
            x1=.84,
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )])
                 

fig.show()

In [21]:
addresses = top_ten[1:6]

zipc = []
long = []
lat = []

for i in addresses:
    geolocator = Nominatim(user_agent="peter")
    location = geolocator.geocode(i)
    latitude = location.latitude
    longitude = location.longitude
    long.append(longitude)
    lat.append(latitude)
    zipc.append(i)

    
time.sleep(4)

addresses = top_ten[6:11]   
for i in addresses:
    geolocator = Nominatim(user_agent="peter")
    location = geolocator.geocode(i)
    latitude = location.latitude
    longitude = location.longitude
    long.append(longitude)
    lat.append(latitude)
    zipc.append(i)
        


In [21]:
#atl_coord = pd.DataFrame(columns=['Zipcode', 'Latitude', 'Longitude'])
#atl_coord.Zipcode = zipc
#atl_coord.Latitude = lat
#atl_coord.Longitude = long

#atl_coord.to_csv('atl_coord.csv', index = False)
atl_coord = pd.read_csv('atl_coord.csv')
atl_coord.head()

,Zipcode,Latitude,Longitude
0,30310,33.745139,-84.427804
1,30314,33.750838,-84.445932
2,30311,33.723504,-84.470770
3,30315,33.734978,-84.383185
4,30344,33.706159,-84.457296


In [22]:
address = 'Atlanta, Georgia'

geolocator = Nominatim(user_agent="pete")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Atlanta are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Atlanta are 33.7490987, -84.3901849.


In [23]:
map_atlanta = folium.Map(location=[latitude, longitude], zoom_start=10)

for latitude, longitude, zipcode in zip(atl_coord.Latitude, atl_coord.Longitude, atl_coord.Zipcode):
    label = '{}'.format(zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_atlanta)

map_atlanta

In [207]:
map_atlanta.save('map_atlanta_1.html')

In [96]:
CLIENT_ID = 'CCBMXEF4ITVUNF0RDKXYHAMX5F55TLG3DCMYIE3DH44AQBCY' # your Foursquare ID
CLIENT_SECRET = 'UXRV1ZBEKD1DCQZIZIDC3L1YMHNNWQIVZC4XVISBTKKZAKTW' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CCBMXEF4ITVUNF0RDKXYHAMX5F55TLG3DCMYIE3DH44AQBCY
CLIENT_SECRET:UXRV1ZBEKD1DCQZIZIDC3L1YMHNNWQIVZC4XVISBTKKZAKTW


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
atl_venues = getNearbyVenues(names= atl_coord.Zipcode,
                                   latitudes= atl_coord.Latitude,
                                   longitudes= atl_coord.Longitude
                                  )


30310
30314
30311
30315
30344
30354
30318
30291
30316
30331


In [24]:
#atl_venues.to_csv('atl_venues.csv', index = False)
atl_venues = pd.read_csv('atl_venues.csv')
atl_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,30310,33.745139,-84.427804,West End Park,33.741926,-84.424973,Park
1,30310,33.745139,-84.427804,Tassili's Raw Reality,33.738453,-84.422394,Vegetarian / Vegan Restaurant
2,30310,33.745139,-84.427804,Lean Draft House,33.738184,-84.430623,Beer Bar
3,30310,33.745139,-84.427804,Ray Charles Performing Arts Center - Morehouse...,33.744034,-84.416824,Concert Hall
4,30310,33.745139,-84.427804,Atlanta BeltLine Corridor over MLK,33.750790,-84.426363,Trail


In [25]:
print('There are {} uniques categories.'.format(len(atl_venues['Venue Category'].unique())))

There are 103 uniques categories.


In [26]:

atl_onehot = pd.get_dummies(atl_venues[['Venue Category']], prefix="", prefix_sep="")


atl_onehot['Zipcode'] = atl_venues['Neighborhood'] 


atl_onehot.shape

(262, 104)

In [71]:
atl_grouped = atl_onehot.groupby('Zipcode').sum().reset_index()

In [72]:
atl_grouped.head()

,Zipcode,Accessories Store,American Restaurant,Antique Shop,Art Museum,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,...,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Zoo,Zoo Exhibit
0,30291,0,1,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
1,30310,0,0,0,1,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
2,30311,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30314,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,30315,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,4


In [73]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Zipcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


zipcode_venues_sorted = pd.DataFrame(columns=columns)
zipcode_venues_sorted['Zipcode'] = atl_grouped['Zipcode']

for ind in np.arange(atl_grouped.shape[0]):
    zipcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(atl_grouped.iloc[ind, :], num_top_venues)

zipcode_venues_sorted.head()

,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,30291,Bank,Mobile Phone Shop,Hotel,Sandwich Place,Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Discount Store,Caribbean Restaurant,Seafood Restaurant
1,30310,Trail,Park,Southern / Soul Food Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Beer Bar,Tennis Court,Cosmetics Shop,Paper / Office Supplies Store,Donut Shop
2,30311,Pharmacy,Caribbean Restaurant,Dry Cleaner,Business Service,Park,Seafood Restaurant,Discount Store,Fast Food Restaurant,Nature Preserve,Chinese Restaurant
3,30314,Trail,Gas Station,Park,Fried Chicken Joint,Chinese Restaurant,Beer Bar,Pharmacy,Cemetery,Discount Store,Donut Shop
4,30315,Zoo Exhibit,Coffee Shop,Brewery,Burger Joint,Pizza Place,Greek Restaurant,Beer Store,Playground,Pet Store,Parking


In [75]:
atl_grouped['Zipcode'] = atl_grouped.Zipcode.astype(str)

num_top_venues = 10
venue = []
freq = []
zipcode = []


for hood in atl_grouped['Zipcode']:
    print("----"+hood+"----")
    temp = atl_grouped[atl_grouped['Zipcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    temp['zipcode'] = hood
    g_data = temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues)
    for i in g_data.venue.values:
        venue.append(i)
    for i in g_data.freq.values:
        freq.append(i)
    for i in g_data.zipcode.values:
        zipcode.append(i)
    
   
    
top_ten_venues = pd.DataFrame()
top_ten_venues['Zipcode'] = zipcode
top_ten_venues['Venues'] = venue
top_ten_venues['Freq'] = freq


----30291----
----30310----
----30311----
----30314----
----30315----
----30316----
----30318----
----30331----
----30344----
----30354----


In [76]:
top_ten_venues.head()

,Zipcode,Venues,Freq
0,30291,Bank,3.0
1,30291,Fast Food Restaurant,2.0
2,30291,Hotel,2.0
3,30291,Sandwich Place,2.0
4,30291,Mobile Phone Shop,2.0


In [77]:
top_ten_grouped = top_ten_venues.groupby('Venues')['Freq'].sum().sort_values(ascending = False).reset_index()
fig = px.bar(top_ten_grouped, x = 'Venues', y = 'Freq')

fig.update_layout(title = '<b>Highest Frequency Venues</b>', width = 1200)
fig.update_xaxes(title = '<b>Venues</b>',tickangle = 45)
fig.update_yaxes(title = '<b>Frequency</b>')

fig.show()

In [34]:
trail_list = list(top_ten_venues[top_ten_venues.Venues == 'Trail'].Zipcode.values.astype(int))
trail_freq = list(top_ten_venues[top_ten_venues.Venues == 'Trail'].Freq.values)
trail_list

[30310, 30314, 30354]

In [35]:
trail_venues = top_ten_venues[top_ten_venues.Venues == 'Trail']
trail_venues

,Zipcode,Venues,Freq
10,30310,Trail,7.0
30,30314,Trail,5.0
90,30354,Trail,1.0


In [36]:
map_atlanta = folium.Map(location=[latitude, longitude], zoom_start=10)



for latitude, longitude, zipcode in zip(atl_coord.Latitude, atl_coord.Longitude, atl_coord.Zipcode):
    if zipcode in trail_list:
        label = '{}'.format(zipcode)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [latitude, longitude],
            radius=5,
            popup=label,
            color= 'green',
            fill= True,
            fill_color='green',
            fill_opacity=0.7,
            parse_html=False).add_to(map_atlanta)
    else:
        label = '{}'.format(zipcode)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [latitude, longitude],
            radius=5,
            popup=label,
            color= 'red',
            fill= True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(map_atlanta)

map_atlanta

In [210]:
trails = atl_venues[atl_venues['Venue Category'] =='Trail']
trails

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,30310,33.745139,-84.427804,Atlanta BeltLine Corridor over MLK,33.750790,-84.426363,Trail
6,30310,33.745139,-84.427804,Atlanta BeltLine Langhorn to Westview,33.750558,-84.427511,Trail
7,30310,33.745139,-84.427804,Atlanta BeltLine Corridor under Lucile Ave,33.742318,-84.433945,Trail
11,30310,33.745139,-84.427804,Atlanta BeltLine Corridor under Ralph David Ab...,33.739007,-84.433123,Trail
20,30310,33.745139,-84.427804,Atlanta BeltLine Corridor at Lena St,33.756441,-84.424909,Trail
23,30310,33.745139,-84.427804,Atlanta BeltLine Corridor under I-20,33.751341,-84.436525,Trail
27,30310,33.745139,-84.427804,Atlanta BeltLine Corridor under Lawton,33.732403,-84.423537,Trail
32,30314,33.750838,-84.445932,Atlanta BeltLine Corridor under I-20,33.751341,-84.436525,Trail
34,30314,33.750838,-84.445932,Atlanta BeltLine Corridor under Lucile Ave,33.742318,-84.433945,Trail
36,30314,33.750838,-84.445932,Atlanta BeltLine Corridor over MLK,33.750790,-84.426363,Trail


In [89]:
map_atlanta = folium.Map(location=[latitude, longitude], zoom_start=10)


for latitude, longitude, zipcode in zip(trails['Venue Latitude'], trails['Venue Longitude'], trails['Venue']):
    label = '{}'.format(zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
            [latitude, longitude],
            radius=5,
            popup=label,
            color= 'red',
            fill= True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(map_atlanta)
 
map_atlanta

### Looking at other major cities with developed trail economies: Ernest E Debs Regional Park, Arboretum Outer Loop Trail, The 606, White Rock Lake Trail

In [151]:
six_oh_six = '1801 N California Ave, Chicago, IL'
white_rock = '8300 E Lawther Dr., Dallas, TX'
ernest = '511 W 22nd St., New York, NY 10011'

In [152]:
addresses = [six_oh_six, white_rock, ernest]

zipc = []
long = []
lat = []

for i in addresses:
    geolocator = Nominatim(user_agent="peter")
    location = geolocator.geocode(i)
    latitude = location.latitude
    longitude = location.longitude
    long.append(longitude)
    lat.append(latitude)
    zipc.append(i)
    
developed_trails = pd.DataFrame(columns=['Zipcode', 'Latitude', 'Longitude'])
developed_trails.Zipcode = zipc
developed_trails.Latitude = lat
developed_trails.Longitude = long


In [153]:
developed_trails.head()

,Zipcode,Latitude,Longitude
0,"1801 N California Ave, Chicago, IL",41.914129,-87.696991
1,"8300 E Lawther Dr., Dallas, TX",32.841340,-96.714399
2,"511 W 22nd St., New York, NY 10011",40.747207,-74.004868


In [156]:
def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [157]:
trail_venues = getNearbyVenues(names= developed_trails.Zipcode,
                                   latitudes= developed_trails.Latitude,
                                   longitudes= developed_trails.Longitude
                                  )


1801 N California Ave, Chicago, IL
8300 E Lawther Dr., Dallas, TX
511 W 22nd St., New York, NY 10011


In [158]:
trail_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"1801 N California Ave, Chicago, IL",41.914129,-87.696991,"Table, Donkey and Stick",41.917657,-87.696074,New American Restaurant
1,"1801 N California Ave, Chicago, IL",41.914129,-87.696991,Middle Brow Bungalow,41.917664,-87.698777,Brewery
2,"1801 N California Ave, Chicago, IL",41.914129,-87.696991,Twisted Scissors,41.917711,-87.694103,Salon / Barbershop
3,"1801 N California Ave, Chicago, IL",41.914129,-87.696991,Katherine Anne Confections,41.917475,-87.696483,Dessert Shop
4,"1801 N California Ave, Chicago, IL",41.914129,-87.696991,The 606 / Midpoint,41.913855,-87.691507,Trail


In [159]:

trail_onehot = pd.get_dummies(trail_venues[['Venue Category']], prefix="", prefix_sep="")


trail_onehot['Zipcode'] = trail_venues['Neighborhood'] 


trail_onehot.shape

(90, 60)

In [160]:
trail_grouped = trail_onehot.groupby('Zipcode').sum().reset_index()

trail_grouped['Zipcode'] = trail_grouped.Zipcode.astype(str)

num_top_venues = 10
venue = []
freq = []
zipcode = []


for hood in trail_grouped['Zipcode']:
    print("----"+hood+"----")
    temp = trail_grouped[trail_grouped['Zipcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    temp['zipcode'] = hood
    g_data = temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues)
    for i in g_data.venue.values:
        venue.append(i)
    for i in g_data.freq.values:
        freq.append(i)
    for i in g_data.zipcode.values:
        zipcode.append(i)
    
   
    
top_ten_venues = pd.DataFrame()
top_ten_venues['Zipcode'] = zipcode
top_ten_venues['Venues'] = venue
top_ten_venues['Freq'] = freq

----1801 N California Ave, Chicago, IL----
----511 W 22nd St., New York, NY 10011----
----8300 E Lawther Dr., Dallas, TX----


In [161]:
top_ten_venues.head()

,Zipcode,Venues,Freq
0,"1801 N California Ave, Chicago, IL",Brewery,3.0
1,"1801 N California Ave, Chicago, IL",Coffee Shop,3.0
2,"1801 N California Ave, Chicago, IL",Trail,2.0
3,"1801 N California Ave, Chicago, IL",Pizza Place,2.0
4,"1801 N California Ave, Chicago, IL",Park,2.0


In [162]:
top_ten_grouped = top_ten_venues.groupby('Venues')['Freq'].sum().sort_values(ascending = False).reset_index()
fig = px.bar(top_ten_grouped, x = 'Venues', y = 'Freq')

fig.update_layout(title = '<b>Highest Frequency Venues</b>', width = 1200)
fig.update_xaxes(title = '<b>Venues</b>',tickangle = 45)
fig.update_yaxes(title = '<b>Frequency</b>')

fig.show()

In [169]:
trail_venues[trail_venues['Venue Category'] == 'Bakery']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
46,"8300 E Lawther Dr., Dallas, TX",32.841340,-96.714399,Nothing Bundt Cakes,32.832815,-96.702701,Bakery
55,"8300 E Lawther Dr., Dallas, TX",32.841340,-96.714399,Unrefined Bakery,32.845981,-96.710815,Bakery
70,"511 W 22nd St., New York, NY 10011",40.747207,-74.004868,Fabrique Bakery,40.740717,-74.004722,Bakery
75,"511 W 22nd St., New York, NY 10011",40.747207,-74.004868,Aux Merveilleux de Fred,40.738334,-74.004114,Bakery


In [209]:
trail_venues.to_csv('trail_venues.csv')

In [237]:
trails = trails.iloc[[4,7],:]

In [238]:
def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [239]:
trail_venues_2 = getNearbyVenues(names= trails.Venue,
                                   latitudes= trails['Venue Latitude'],
                                   longitudes= trails['Venue Longitude']
                                  )

Atlanta BeltLine Corridor at Lena St
Atlanta BeltLine Corridor under I-20


In [240]:
trail_venues_2.to_csv('final_venues.csv', index = False)

In [241]:

final_onehot = pd.get_dummies(trail_venues_2[['Venue Category']], prefix="", prefix_sep="")


final_onehot['Zipcode'] = trail_venues_2['Neighborhood'] 


final_grouped = final_onehot.groupby('Zipcode').sum().reset_index()

In [242]:


num_top_venues = 10
venue = []
freq = []
zipcode = []


for hood in final_grouped['Zipcode']:
    print("----"+hood+"----")
    temp = final_grouped[final_grouped['Zipcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    temp['zipcode'] = hood
    g_data = temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues)
    for i in g_data.venue.values:
        venue.append(i)
    for i in g_data.freq.values:
        freq.append(i)
    for i in g_data.zipcode.values:
        zipcode.append(i)
    
   
    
top_ten_venues = pd.DataFrame()
top_ten_venues['Zipcode'] = zipcode
top_ten_venues['Venues'] = venue
top_ten_venues['Freq'] = freq

----Atlanta BeltLine Corridor at Lena St----
----Atlanta BeltLine Corridor under I-20----


In [243]:
top_ten_grouped = top_ten_venues.groupby('Venues')['Freq'].sum().sort_values(ascending = False).reset_index()
fig = px.bar(top_ten_grouped, x = 'Venues', y = 'Freq')

fig.update_layout(title = '<b>Highest Frequency Venues</b>', width = 1200)
fig.update_xaxes(title = '<b>Venues</b>',tickangle = 45)
fig.update_yaxes(title = '<b>Frequency</b>')

fig.show()